# Traiter la donnée -- Dataframes


Pour le réaliser sans installation, depuis un navigateur :
<a href="https://colab.research.google.com/github/eddes/INSA/blob/main/python/tuto_dataframe.ipynb"> ça se passe ici<a>


Dans de nombreux des domaines, la donnée disponible est fournie dans des fichiers texte, où les valeurs des différentes grandeurs sont séparées par des virgules (vos tableurs favoris exportent parfois en `.csv` - acronyme de _comma separated values_).
    
Une librairie particulièrement bien adaptée au traitement de tels fichiers est `pandas`. Dans ce qui suit, nous allons eb apprendre les bases.
    
## Ouverture et affichage de base

Ouvrons un des fichiers de données météorologiques mesurées près de la ville de Strasbourg en 2023.

In [39]:
import pandas as pd
chemin='./src/meteo_Strasbourg_2023.csv'
donnee = pd.read_csv(chemin, index_col=0) # on precise d'ores et deja que l'index est en colonne "0"

Pour afficher quelques stats de base sur les données :

In [40]:
donnee.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,1848.000000,1848.000000,1848.000000,1848.000000,608.0,1848.000000,1848.000000,1848.000000,1848.000000,0.0,1848.000000
mean,19.921916,10.926840,59.792208,0.052489,0.0,170.846861,11.822294,21.776136,1016.550866,NaN,4.145563
std,5.818102,3.669961,19.372118,0.420076,0.0,129.132250,6.827365,9.394996,3.602004,NaN,2.867574
min,4.500000,2.100000,21.000000,0.000000,0.0,0.000000,0.000000,3.700000,1007.700000,NaN,1.000000
25%,15.575000,8.500000,44.000000,0.000000,0.0,30.000000,6.000000,14.800000,1014.000000,NaN,4.000000
50%,19.200000,10.500000,60.000000,0.000000,0.0,180.000000,11.000000,20.400000,1016.300000,NaN,4.000000
75%,24.500000,13.600000,75.000000,0.000000,0.0,300.000000,16.600000,27.800000,1018.700000,NaN,4.000000
max,36.000000,21.000000,100.000000,9.000000,0.0,360.000000,35.300000,57.000000,1026.100000,NaN,25.000000


On peut donc, pour chaque colonne, lire les min/max ainsi que les percentiles de la série.

Ensuite, il est pratique de connaître le nom des données stockées, pour pouvoir les appeler séparément :

In [41]:
donnee.keys()

Index(['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun', 'coco'],
      dtype='object')

On constate que le temps `time` et la température `temp` sont stockées. On va pouvoir y faire appel simplement par leur nom avec une commande du style :

In [42]:
donnee['temp']

time
2023-05-01 00:00:00    10.3
2023-05-01 01:00:00     9.8
2023-05-01 02:00:00     9.8
2023-05-01 03:00:00     9.8
2023-05-01 04:00:00    10.0
                       ... 
2023-07-16 19:00:00    21.0
2023-07-16 20:00:00    20.0
2023-07-16 21:00:00    19.6
2023-07-16 22:00:00    18.0
2023-07-16 23:00:00    17.0
Name: temp, Length: 1848, dtype: float64

On notera la présence d'un index (colonne de gauche qui est affichée). Cet index est commun à toutes les données stockées, lorsqu'il y en a plusieurs. Il peut aussi être un temps (heure/min/sec j/m/a) ce qui se révèlera particulièrement pratique par la suite.

Si on souhaite uniquement en prendre les valeurs on utilise le suffixe `values` :

In [43]:
T = donnee['temp'].values
print(T)

[10.3  9.8  9.8 ... 19.6 18.  17. ]


## Sélection de données avec pandas

Il est parfois pratique de connaître/isoler rapidement les valeurs en-dessous ou au-dessus d'un certain seuil. Supposons qu'on veuille connaitre les instants où la température est supérieure à 26°C :

In [44]:
fait_chaud = donnee[ donnee['temp'] > 26]
print(fait_chaud)

                     temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  \
time                                                                           
2023-05-21 15:00:00  26.5  14.3  47.0   0.0   0.0   20.0  16.6  41.0  1010.7   
2023-05-21 16:00:00  26.4  13.9  46.0   0.0   NaN   20.0  18.4  25.9  1011.0   
2023-05-22 11:00:00  26.3  15.4  51.0   0.0   NaN   40.0   9.4  20.4  1012.5   
2023-05-22 12:00:00  27.2  15.2  48.0   0.0   0.0  120.0   3.6  24.0  1012.5   
2023-05-22 13:00:00  26.8  15.5  50.0   0.0   NaN  280.0  13.0  24.1  1012.1   
...                   ...   ...   ...   ...   ...    ...   ...   ...     ...   
2023-07-15 14:00:00  30.0  17.1  46.0   0.0   NaN  170.0  15.0  29.6  1010.0   
2023-07-15 15:00:00  30.6  19.9  53.0   0.0   0.0   20.0  15.0  31.0  1008.6   
2023-07-15 16:00:00  29.0  19.0  55.0   0.0   NaN  360.0  19.0  40.8  1009.0   
2023-07-15 17:00:00  29.0  19.0  55.0   0.0   NaN   10.0  20.0  33.3  1009.0   
2023-07-15 18:00:00  26.3  18.4  62.0   

Il y a donc 309 valeurs avec une température supérieure à 26.

Pour pouvoir identifier s'il s'agit du jour ou de la nuit, on peut utiliser la colonne `time` en s'assurant de convertir l'index en un format permettant de gérer les opérations sur le temps (un horodatage en quelque sorte - voir plus bas les différentes techniques pour permettre la reconnaissance de l'horodatage des données lors de la lecture du `.csv`) :

In [48]:
donnee.index = pd.to_datetime(donnee.index) # ici on convertit l'index en datetime
print(donnee.index)

DatetimeIndex(['2023-05-01 00:00:00', '2023-05-01 01:00:00',
               '2023-05-01 02:00:00', '2023-05-01 03:00:00',
               '2023-05-01 04:00:00', '2023-05-01 05:00:00',
               '2023-05-01 06:00:00', '2023-05-01 07:00:00',
               '2023-05-01 08:00:00', '2023-05-01 09:00:00',
               ...
               '2023-07-16 14:00:00', '2023-07-16 15:00:00',
               '2023-07-16 16:00:00', '2023-07-16 17:00:00',
               '2023-07-16 18:00:00', '2023-07-16 19:00:00',
               '2023-07-16 20:00:00', '2023-07-16 21:00:00',
               '2023-07-16 22:00:00', '2023-07-16 23:00:00'],
              dtype='datetime64[ns]', name='time', length=1848, freq=None)


On constate que le format de donnée de l'index `dtype` correspond désormais à un `datetime`. Ceci va nous faciliter la tâche pour trier la donnée. Si l'on souhaite connaître les moments de l'année 2023 où il a fait plus de 22°C après 21h à Strasbourg, on peut filtrer la donnée comme suit :

In [49]:
nuit_chaud = donnee[ (donnee.index.hour > 21) & (donnee['temp']>22) ]
print(df_chaud)

                     temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  \
time                                                                           
2023-05-01 23:00:00  11.8   8.1  78.0   0.0   NaN  310.0   7.6  11.1  1018.9   
2023-05-02 23:00:00  10.8   5.1  68.0   0.0   NaN   30.0  22.3  16.7  1025.8   
2023-05-03 23:00:00  11.4   5.9  69.0   0.0   NaN   20.0   9.4   9.3  1021.3   
2023-05-04 23:00:00  15.7  12.1  79.0   0.0   NaN  250.0   5.4  11.1  1017.7   
2023-05-05 23:00:00  13.7  13.1  96.0   0.0   NaN  290.0   5.4  20.4  1018.5   
...                   ...   ...   ...   ...   ...    ...   ...   ...     ...   
2023-07-12 23:00:00  17.0  11.9  72.0   0.0   NaN  280.0  11.0  13.0  1017.0   
2023-07-13 23:00:00  16.0  11.0  72.0   0.0   NaN  257.0   4.0   9.3  1019.0   
2023-07-14 23:00:00  19.0  15.1  78.0   0.0   NaN  360.0   6.0  11.1  1011.0   
2023-07-15 23:00:00  19.0  18.0  94.0   2.2   NaN  140.0   7.0  29.6  1015.0   
2023-07-16 23:00:00  17.0  12.9  77.0   

###  Fusion de deux dataframe en conservant l'index, les valeurs et en interpolant sur les NaN

Oui, c'est possible et c'est même très pratique : par exemple quand on récupère des données mesurées et qu'on souhaite y adjoindre une mesure sur un autre pas de temps. Un exemple typique est le cas de mesures dans un bâtiment et de la température extérieure, récupérée par exemple avec <a href="https://colab.research.google.com/github/eddes/INSA/blob/main/python/tuto_meteostat.ipynb"> Meteostat<a>.
    
L'exemple qui suit traite de ce cas pratique, avec un fichier **meteo.csv** qui contient des mesures à heures fixes et un fixer **mesure.csv** qui contient des mesures toutes les 15 minutes, sans être calé sur une heure "pile" (ouvrir le fichier pour mieux saisir).

In [50]:
import matplotlib.pyplot as plt
import pandas as pd
# lecture du fichier meteo
df_meteo = pd.read_csv('./src/meteo.csv', index_col=0, header="infer",delimiter=',')
print('quoi dedans ? ', df_meteo.keys())

# on nettoie car seule la premiere colonne 'temp' nous interesse et on ne souhaite pas realiser l'interpolation sur toutes les colonnes
# 'temp' est la grandeur qui nous interesse, c'est la premiere colonne
# on cree donc une liste des colonnes qui vont etre supprimées
cols = range(1, len(df_meteo .keys() ))  
# c'est parti : noter la syntaxe df.drop()
df_meteo.drop(df_meteo .columns[cols], axis=1, inplace=True) 

# on lit les donnees mesurees
df_mesure = pd.read_csv('./src/mesure.csv', index_col=0, header="infer",delimiter=',')

# on va fusionner les deux et surtout les reclasser par index, sinon pas d'interpolation
df_concat = pd.concat([df_mesure,df_meteo]).sort_index()

# puis on interpole lineairement sur les donnees disponibles (il y a plusieurs methodes d'interpolation)
df_interp = df_concat.interpolate(method='linear')

plt.subplot(121)
plt.plot(df_meteo['temp'],'k-o', label='original')
plt.legend()
plt.subplot(122)
plt.plot(df_interp['temp'], 'k--.', label='interpole')
plt.legend()

# au besoin on sauve pour regarder ce qu'il y a dedans
# df_interp.to_csv('./data_interp.csv')
# df_concat.to_csv('./data_concat.csv')

FileNotFoundError: [Errno 2] No such file or directory: './src/meteo.csv'

## Cheat cheat dataframe

Quelques bouts de code utiles...

### Convertir un index de .csv en datetime

#### Plan A

Si le format de date est classique YYYY-MM-DD HH:MM:SS

`donnee.index = pd.to_datetime(donnee.index)`

Si le format de date n'est pas reconnu

`donnee.index = pd.to_datetime(donnee.index, format='%Y-%m-%d %H:%M:%S')`

#### Plan B - avec dates "à la française" 

Dans ce cas le jour apparaît en premier, puis le mois, puis l'année:
`df.index = pd.to_datetime(df.index, dayfirst=True)`

#### Plan C

En comptant sur la chance, on demande à pandas de trouver le format de date pour nous

`dossier='./src/'`

`nom='data.csv`

`df= pd.read_csv(dossier+nom, index_col=0, parse_dates=[0])`

### Suppression de valeurs

#### Les doublons

On supprime par index :
`df.drop_duplicates(inplace=True)`

#### Les colonnes

Il faut définir la liste `cols` ou l'entier `cols` au préalable (identifier les indices des colonnes que l'on souhaite supprimer)

`df.drop( df.columns[cols], axis=1, inplace=True)`

#### Arrondir à deux décimales

`df = df.round({'temperature':2})`

#### Remplacer des valeurs non souhaitées

Par exemple lorsqu'un capteur renvoie trois tirets quand il ne capte plus :

`df['Tair'] = df['Tair'].replace('---', np.nan)`

Ou lorsqu'une case est vide (parfois pratique de remplacer par __not a number__ - np.nan)

`df['Tair'] = df['Tair'].replace([None], np.nan) `

In [ ]:
Beaucoup plus chez https://matbog.github.io/code